In [2]:
import pandas as pd
backtesting_data = pd.read_csv("/Users/giulioelmi/Desktop/kelshi_trading/backtesting/backtesting_data.csv", index_col = 0)
data_start_date = backtesting_data["DATE"].min()
data_end_date = backtesting_data["DATE"].max()
series_ticker = "KXHIGHLAX"
backtesting_data_no_date = backtesting_data.drop(columns = ["DATE", "y", "TMAX_next_d"])

In [3]:
backtesting_data_no_date.head()

,TMAX,TMIN,PRCP,AWND,WDF2,WSF2,year,forecasted_TMAX,doy,dow,...,diurnal_range_lag1,diurnal_range_lag2,diurnal_range_lag3,diurnal_range_lag7,PRCP_lag_1,PRCP_lag_2,PRCP_lag_3,PRCP_lag_4,rolling_3,rolling_7
1520,62,55,0.00,6.49,240.0,23.9,2025,60.044,60,5,...,15.0,36.0,13.0,20.0,0.0,0.0,0.0,0.0,72.333333,70.285714
1521,63,53,0.00,14.99,260.0,35.1,2025,60.872,61,6,...,7.0,15.0,36.0,22.0,0.0,0.0,0.0,0.0,64.666667,69.142857
1522,63,50,0.00,11.18,260.0,29.1,2025,63.284,62,0,...,10.0,7.0,15.0,19.0,0.0,0.0,0.0,0.0,62.666667,68.285714
1523,64,50,0.00,9.17,260.0,19.9,2025,59.162,63,1,...,13.0,10.0,7.0,18.0,0.0,0.0,0.0,0.0,63.333333,67.428571
1524,58,50,0.45,6.49,260.0,25.9,2025,55.202,64,2,...,14.0,13.0,10.0,13.0,0.0,0.0,0.0,0.0,61.666667,66.428571


In [4]:
results = []
from model_copy import make_prediction

best_model_path = "/Users/giulioelmi/Desktop/kelshi_trading/backtesting/best_backtesting.json"
for i in range(len(backtesting_data_no_date)):
    row_df = backtesting_data_no_date.iloc[i:i+1]      # one-row DataFrame
    pred = make_prediction(row_df, best_model_path)

    results.append({
        "adjusted_forecast": pred
    })

results_df = pd.DataFrame(results)
results_df["DATE"] = backtesting_data["DATE"].values

In [5]:
results_df.head()

,adjusted_forecast,DATE
0,60.890589,2025-03-01
1,61.589648,2025-03-02
2,63.795708,2025-03-03
3,59.893102,2025-03-04
4,55.463709,2025-03-05


In [6]:
from get_market_data import make_daily_request_table

req = make_daily_request_table(
    series_ticker=series_ticker,
    start_date=data_start_date,
    end_date=data_end_date,
    target_hhmm="12:00",      # target time within each day         
)

In [7]:
req.head()

,series_ticker,event_ticker,start_time,end_time,target_time,start_ts,end_ts,target_ts,day
0,KXHIGHLAX,KXHIGHLAX-25MAR01,2025-03-01T10:30:00+00:00,2025-03-01T13:30:00+00:00,2025-03-01T12:00:00+00:00,1740825000,1740835800,1740830400,2025-03-01
1,KXHIGHLAX,KXHIGHLAX-25MAR02,2025-03-02T10:30:00+00:00,2025-03-02T13:30:00+00:00,2025-03-02T12:00:00+00:00,1740911400,1740922200,1740916800,2025-03-02
2,KXHIGHLAX,KXHIGHLAX-25MAR03,2025-03-03T10:30:00+00:00,2025-03-03T13:30:00+00:00,2025-03-03T12:00:00+00:00,1740997800,1741008600,1741003200,2025-03-03
3,KXHIGHLAX,KXHIGHLAX-25MAR04,2025-03-04T10:30:00+00:00,2025-03-04T13:30:00+00:00,2025-03-04T12:00:00+00:00,1741084200,1741095000,1741089600,2025-03-04
4,KXHIGHLAX,KXHIGHLAX-25MAR05,2025-03-05T10:30:00+00:00,2025-03-05T13:30:00+00:00,2025-03-05T12:00:00+00:00,1741170600,1741181400,1741176000,2025-03-05


In [8]:
from get_market_data import fetch_daily_candles_from_table

market_prices = fetch_daily_candles_from_table(req)

In [9]:
market_prices.head()

,ticker,threshold_type,threshold,yes_ask,day,event_ticker
0,KXHIGHLAX-25MAR01-T57,T,57.0,0.01,2025-03-01,KXHIGHLAX-25MAR01
1,KXHIGHLAX-25MAR01-B57.5,B,57.5,0.09,2025-03-01,KXHIGHLAX-25MAR01
2,KXHIGHLAX-25MAR01-B59.5,B,59.5,0.30,2025-03-01,KXHIGHLAX-25MAR01
3,KXHIGHLAX-25MAR01-B61.5,B,61.5,0.39,2025-03-01,KXHIGHLAX-25MAR01
4,KXHIGHLAX-25MAR01-B63.5,B,63.5,0.40,2025-03-01,KXHIGHLAX-25MAR01


In [10]:
import numpy as np
import pandas as pd

def add_floor_cap(df, group_cols=("day", "event_ticker")):
    out = df.copy()

    # initialize
    out["floor"] = np.nan
    out["cap"] = np.nan

    # B buckets
    is_B = out["threshold_type"].eq("B")
    out.loc[is_B, "floor"] = out.loc[is_B, "threshold"] - 0.5
    out.loc[is_B, "cap"]   = out.loc[is_B, "threshold"] + 0.5

    is_T = out["threshold_type"].eq("T")

    def _assign_tails(g):
        g = g.copy()
        t_idx = g.index[g["threshold_type"].eq("T")]
        if len(t_idx) == 0:
            return g

        # Use B buckets to infer where tails should be
        b = g[g["threshold_type"].eq("B")]
        if not b.empty:
            min_b_floor = (b["threshold"] - 0.5).min()
            max_b_cap   = (b["threshold"] + 0.5).max()

            for i in t_idx:
                thr = float(g.loc[i, "threshold"])
                if thr <= min_b_floor + 1e-9:      # lower tail (< thr)
                    g.loc[i, "floor"] = np.nan
                    g.loc[i, "cap"]   = thr
                elif thr >= max_b_cap - 1e-9:      # upper tail (>= thr)
                    g.loc[i, "floor"] = thr
                    g.loc[i, "cap"]   = np.nan
                else:
                    # ambiguous (rare): fallback to min/max T inside the group
                    pass
            return g

        # Fallback if there are no B rows: use min/max T in the group
        t_vals = g.loc[t_idx, "threshold"].astype(float)
        t_min, t_max = t_vals.min(), t_vals.max()

        g.loc[t_idx[t_vals.eq(t_min)], "cap"] = t_min
        g.loc[t_idx[t_vals.eq(t_max)], "floor"] = t_max
        return g

    out = out.groupby(list(group_cols), group_keys=False).apply(_assign_tails)
    return out


In [11]:
markets_df = add_floor_cap(market_prices, group_cols=("day", "event_ticker"))

/var/folders/7d/dvvdf5fj3_b9534z4rpkvr440000gn/T/ipykernel_94613/2404498861.py:51: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  out = out.groupby(list(group_cols), group_keys=False).apply(_assign_tails)


In [12]:
markets_df["no_ask"] = 1 - markets_df["yes_ask"]

In [13]:
markets_df.head()

,ticker,threshold_type,threshold,yes_ask,day,event_ticker,floor,cap,no_ask
0,KXHIGHLAX-25MAR01-T57,T,57.0,0.01,2025-03-01,KXHIGHLAX-25MAR01,NaN,57.0,0.99
1,KXHIGHLAX-25MAR01-B57.5,B,57.5,0.09,2025-03-01,KXHIGHLAX-25MAR01,57.0,58.0,0.91
2,KXHIGHLAX-25MAR01-B59.5,B,59.5,0.30,2025-03-01,KXHIGHLAX-25MAR01,59.0,60.0,0.70
3,KXHIGHLAX-25MAR01-B61.5,B,61.5,0.39,2025-03-01,KXHIGHLAX-25MAR01,61.0,62.0,0.61
4,KXHIGHLAX-25MAR01-B63.5,B,63.5,0.40,2025-03-01,KXHIGHLAX-25MAR01,63.0,64.0,0.60


In [14]:
import pandas as pd
import numpy as np
import math

def compute_daily_evs(
    markets_df: pd.DataFrame,
    results_df: pd.DataFrame,
    sigma: float,
    day_col_markets: str = "day",
    day_col_results: str = "DATE",
    mu_col_results: str = "adjusted_forecast",
) -> pd.DataFrame:
    """
    For each day:
      - filter markets_df to that day
      - grab mu (prediction) for that day from results_df
      - compute EV columns via get_ev
      - append into one big dataframe

    Requirements:
      markets_df must include: day, floor, cap, yes_ask
      results_df must include: DATE, forecasted_TMAX
      If markets_df has no_ask missing, it will be created as (100 - yes_ask).
    """

    # -------------------------
    # 0) Basic copies + dtypes
    # -------------------------
    markets = markets_df.copy()
    results = results_df.copy()

    markets[day_col_markets] = pd.to_datetime(markets[day_col_markets]).dt.date
    results[day_col_results] = pd.to_datetime(results[day_col_results]).dt.date

    # -------------------------
    # 1) Ensure required columns
    # -------------------------
    required_markets = {"floor", "cap", "yes_ask", day_col_markets}
    missing_mk = required_markets - set(markets.columns)
    if missing_mk:
        raise ValueError(f"markets_df missing required columns: {sorted(missing_mk)}")

    required_results = {day_col_results, mu_col_results}
    missing_rs = required_results - set(results.columns)
    if missing_rs:
        raise ValueError(f"results_df missing required columns: {sorted(missing_rs)}")

    # numeric safety
    for c in ["floor", "cap", "yes_ask"]:
        markets[c] = pd.to_numeric(markets[c], errors="coerce")

    results[mu_col_results] = pd.to_numeric(results[mu_col_results], errors="coerce")

    # If no_ask is missing, create it.
    # NOTE: This is a simplification (best is to use actual orderbook no_ask if you have it).
    if "no_ask" not in markets.columns:
        markets["no_ask"] = (100.0 - markets["yes_ask"]).round(2)
    else:
        markets["no_ask"] = pd.to_numeric(markets["no_ask"], errors="coerce")

    # keep only rows where we can price
    markets = markets.dropna(subset=[day_col_markets, "yes_ask", "no_ask"])

    # -------------------------
    # 2) Build a day -> mu map
    # -------------------------
    # if multiple rows per day, take the last non-null (or you can change to mean)
    day_mu = (
        results.dropna(subset=[mu_col_results])
               .sort_values(day_col_results)
               .drop_duplicates(subset=[day_col_results], keep="last")
               .set_index(day_col_results)[mu_col_results]
    )

    # -------------------------
    # 3) Loop days, compute EV, append
    # -------------------------
    outputs = []
    for day, mk_day in markets.groupby(day_col_markets, sort=True):
        if day not in day_mu.index:
            # no prediction for this day -> skip (or keep with NaNs if you prefer)
            continue

        mu = float(day_mu.loc[day])
        out_day = get_ev(mk_day, mu=mu, sigma=float(sigma))
        out_day["mu"] = mu
        out_day["sigma"] = float(sigma)
        outputs.append(out_day)

    if not outputs:
        return pd.DataFrame()

    return pd.concat(outputs, axis=0, ignore_index=True)


In [15]:
SIGMA = 2.5324872296670837
from model_copy import get_ev
ev_df = compute_daily_evs(
    markets_df=markets_df,
    results_df=results_df,
    sigma=SIGMA,  
    day_col_markets="day",
    day_col_results="DATE",
    mu_col_results="adjusted_forecast",
)



In [16]:
ev_df.head()

,ticker,threshold_type,threshold,yes_ask,day,event_ticker,floor,cap,no_ask,p_yes,p_no,max_yes_cents,max_no_cents,edge_yes_cents,edge_no_cents,buy_yes,buy_no,mu,sigma
0,KXHIGHLAX-25MAR01-T57,T,57.0,0.01,2025-03-01,KXHIGHLAX-25MAR01,NaN,57.0,0.99,0.062236,0.937764,0.06,0.94,0.05,-0.05,True,False,60.890589,2.532487
1,KXHIGHLAX-25MAR01-B57.5,B,57.5,0.09,2025-03-01,KXHIGHLAX-25MAR01,57.0,58.0,0.91,0.064615,0.935385,0.06,0.94,-0.03,0.03,False,True,60.890589,2.532487
2,KXHIGHLAX-25MAR01-B59.5,B,59.5,0.30,2025-03-01,KXHIGHLAX-25MAR01,59.0,60.0,0.70,0.134872,0.865128,0.13,0.87,-0.17,0.17,False,True,60.890589,2.532487
3,KXHIGHLAX-25MAR01-B61.5,B,61.5,0.39,2025-03-01,KXHIGHLAX-25MAR01,61.0,62.0,0.61,0.152103,0.847897,0.15,0.85,-0.24,0.24,False,True,60.890589,2.532487
4,KXHIGHLAX-25MAR01-B63.5,B,63.5,0.40,2025-03-01,KXHIGHLAX-25MAR01,63.0,64.0,0.60,0.092680,0.907320,0.09,0.91,-0.31,0.31,False,True,60.890589,2.532487


In [17]:
df_max = ev_df.loc[ev_df.groupby("event_ticker")["edge_no_cents"].idxmax()]

In [36]:
df_max_final = df_max[["day", "threshold","edge_no_cents", "floor", "cap", "no_ask"]]


In [37]:
df_max_final = df_max_final.rename(columns = {"day": "DATE"})
df_max_final["DATE"] = pd.to_datetime(df_max_final["DATE"])

In [38]:
temperature = backtesting_data[["DATE", "TMAX"]]
temperature["DATE"] = pd.to_datetime(temperature["DATE"])

/var/folders/7d/dvvdf5fj3_b9534z4rpkvr440000gn/T/ipykernel_94613/2160792296.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temperature["DATE"] = pd.to_datetime(temperature["DATE"])


In [40]:
df_pnl = df_max_final.merge(temperature, on = "DATE")
df_pnl["fee"] = 0.02

In [41]:
df_pnl


,DATE,threshold,edge_no_cents,floor,cap,no_ask,TMAX,fee
0,2025-04-01,66.5,0.48,66.0,67.0,0.50,63,0.02
1,2025-04-02,62.5,0.37,62.0,63.0,0.48,63,0.02
2,2025-04-03,61.5,0.37,61.0,62.0,0.55,63,0.02
3,2025-04-04,65.5,0.60,65.0,66.0,0.40,64,0.02
4,2025-04-05,66.5,0.45,66.0,67.0,0.54,75,0.02
...,...,...,...,...,...,...,...,...
147,2025-05-27,71.5,0.43,71.0,72.0,0.52,72,0.02
148,2025-05-28,68.5,0.55,68.0,69.0,0.30,67,0.02
149,2025-05-29,69.5,0.54,69.0,70.0,0.44,69,0.02
150,2025-05-30,73.5,0.42,73.0,74.0,0.44,76,0.02


In [42]:
#if TMAX is not equal to the floor or the cap, we put 1, otherwise 0.
df_pnl["outcome"] = ((df_pnl["TMAX"] != df_pnl["floor"]) & 
                     (df_pnl["TMAX"] != df_pnl["cap"])).astype(int)


In [80]:
df_pnl.head(50)


,DATE,threshold,edge_no_cents,floor,cap,no_ask,TMAX,fee,outcome,pnl_1_contract
0,2025-04-01,66.5,0.48,66.0,67.0,0.50,63,0.02,1,0.48
1,2025-04-02,62.5,0.37,62.0,63.0,0.48,63,0.02,0,-0.50
2,2025-04-03,61.5,0.37,61.0,62.0,0.55,63,0.02,1,0.43
3,2025-04-04,65.5,0.60,65.0,66.0,0.40,64,0.02,1,0.58
4,2025-04-05,66.5,0.45,66.0,67.0,0.54,75,0.02,1,0.44
5,2025-04-06,73.0,0.64,73.0,NaN,0.35,77,0.02,1,0.63
6,2025-04-07,71.0,0.46,71.0,NaN,0.50,69,0.02,1,0.48
7,2025-04-08,67.5,0.38,67.0,68.0,0.48,68,0.02,0,-0.50
8,2025-04-09,72.5,0.41,72.0,73.0,0.55,72,0.02,0,-0.57
9,2025-04-10,71.5,0.24,71.0,72.0,0.60,71,0.02,0,-0.62


In [71]:
#calculate pnl per day if we buy one contract of the no with the highet edge
df_pnl["pnl_1_contract"] = df_pnl["outcome"] - df_pnl["no_ask"] - df_pnl["fee"]
profit = df_pnl["pnl_1_contract"].sum()
capital_deployed = df_pnl["no_ask"].sum()
mean = df_pnl["pnl_1_contract"].mean()
sd = df_pnl["pnl_1_contract"].std()
average_edge = df_pnl["edge_no_cents"].mean()
percent_right = round(df_pnl["outcome"].sum()/len(df_pnl)*100)

In [72]:
print("total pnl: ", "$", profit)
print("total capital invested: ", "$", capital_deployed)
print("return on capital: ", df_pnl["no_ask"].sum())
print("% correct guesses: ", percent_right, "%")
print("average edge: ", average_edge)


total pnl:  $ 15.949999999999996
total capital invested:  $ 66.01
return on capital:  66.01
% correct guesses:  56 %
average edge:  0.4627631578947369


In [76]:
#what if we only bet if the edge is higher than a threshold?
threshold = 0.35
df_pnl_1 = df_pnl[df_pnl["edge_no_cents"] > threshold]
df_pnl_1["pnl_threshold"] = df_pnl_1["outcome"] - df_pnl_1["no_ask"] - df_pnl_1["fee"]


/var/folders/7d/dvvdf5fj3_b9534z4rpkvr440000gn/T/ipykernel_94613/3646912346.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pnl_1["pnl_threshold"] = df_pnl_1["outcome"] - df_pnl_1["no_ask"] - df_pnl_1["fee"]


In [77]:
profit_1 = df_pnl_1["pnl_1_contract"].sum()
capital_deployed_1 = df_pnl_1["no_ask"].sum()
mean_1 = df_pnl_1["pnl_1_contract"].mean()
sd_1 = df_pnl_1["pnl_1_contract"].std()
average_edge_1 = df_pnl_1["edge_no_cents"].mean()
percent_right_1 = round(df_pnl_1["outcome"].sum()/len(df_pnl_1)*100)

In [78]:
print("total pnl: ", "$", profit_1)
print("total capital invested: ", "$", capital_deployed_1)
print("return on capital: ", df_pnl_1["no_ask"].sum())
print("% correct guesses: ", percent_right_1, "%")
print("average edge: ", average_edge_1)

total pnl:  $ 14.119999999999997
total capital invested:  $ 51.36
return on capital:  51.36
% correct guesses:  54 %
average edge:  0.4942063492063492
